In [1]:
pwd

'/Users/jisookwon/Documents/크롤링/부킹닷컴'

In [2]:
# Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

In [7]:
# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 7000)")
    time.sleep(1)
scroll_down(driver)

In [6]:
# Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome("/Users/jisookwon/Documents/chromedriver")  # 윈도우는 "chromedriver.exe"
url = "https://www.booking.com/searchresults.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=e19b301d9f2ba429173c4b6aaa971bcd&tmpl=searchresults&checkin_month=5&checkin_monthday=15&checkin_year=2021&checkout_month=5&checkout_monthday=16&checkout_year=2021&class_interval=1&dest_id=4170&dest_type=region&dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=d60890e0e8ae0830&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_all=0&ssb=empty&sshis=0&top_ufis=1&rows=25\
&offset=0"
driver.get(url)
time.sleep(2)

# 데이터 크롤링

In [8]:
# Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome("/Users/jisookwon/Documents/chromedriver")  # 윈도우는 "chromedriver.exe"
url = "https://www.booking.com/searchresults.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=e19b301d9f2ba429173c4b6aaa971bcd&tmpl=searchresults&checkin_month=5&checkin_monthday=15&checkin_year=2021&checkout_month=5&checkout_monthday=16&checkout_year=2021&class_interval=1&dest_id=4170&dest_type=region&dtdisc=0&from_sf=1&group_adults=1&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=region&room1=A&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=d60890e0e8ae0830&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_all=0&ssb=empty&sshis=0&top_ufis=1&rows=25\
&offset=0"
driver.get(url)
time.sleep(2)

hotel_list = []
url_list=[]
rating_list = []
price_list = []
room_list = []

print("크롤링 성공: ")
print('hotel', 'rating', 'price', 'room', 'url')

page = 20   # 크롤링 원하는 페이지 수
for i in range(0, page):
    
    # 호텔 이름
    hotel_raw = driver.find_elements_by_css_selector('.sr-hotel__name')
    for hotel in hotel_raw:
        hotel = hotel.text
        hotel_list.append(hotel)
    # hotel_list = [ x for x in hotel_list if not x =='']
    
    # 별점 가져오기
    rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
    for rating in rating_raw:
        if '\n' in rating.text:
            rating = rating.text.split('\n')[0]
        else:
            rating = rating.text
        rating_list.append(rating)
    
    # 가격 가져오기
    price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
    for price in price_raw:
        if price.text.split('\n') != ['']:
            price = price.text.split('\n')
            price = int([ x for x in price if x != '1박, 성인 1명'][0].split(' ')[-1].replace('₩', '').replace(',', ''))
            price_list.append(price)
    price_list = [x for x in price_list if not x =='']
    
    # 방 정보
    room_raw = driver.find_elements_by_css_selector(".room_link")
    for room in room_raw:
        room_list.append(room.text.split('\n')[0].replace(' -','').strip())
    room_list = [ x for x in room_list if not x =='']
    
    # url
    url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")
    for url in url_raw:
        url = url.get_attribute('href')
        url_list.append(url)
    time.sleep(1)
    # url_list = [ x for x in url_list if not x =='']
    
    # 크롤링 성공한 갯수
    print(len(hotel_list),len(rating_list),len(price_list),len(room_list),len(url_list))
    
    # 다음 페이지 이동
    driver.find_element_by_css_selector('.bk-icon.-iconset-navarrow_right.bui-pagination__icon').click()
    time.sleep(3)
    scroll_down(driver)
    
print("크롤링 완료")

크롤링 성공: 
hotel rating price room url
25 25 0 0 25
50 50 0 0 50
75 75 0 0 75
100 100 0 0 100
125 125 0 0 125
150 150 0 0 150
175 175 0 0 175
200 200 0 0 200
225 225 0 0 225
250 250 0 0 250
275 275 0 0 275
300 300 0 0 300
325 325 0 0 325
350 350 0 0 350
375 375 0 0 375
400 400 0 0 400
425 425 0 0 425
450 450 0 0 450
475 475 0 0 475
500 500 0 0 500
크롤링 완료


In [9]:
print(len(hotel_list))
print(len(room_list))
print(len(price_list))
print(len(rating_list))
print(len(url_list))

500
0
0
500
500


In [12]:
# 데이터 프레임으로 만들기
df = pd.DataFrame({'hotel':hotel_list, 'rating':rating_list, 'room':room_list, 'url':url_list, 'price(won)':price_list})
df

ValueError: arrays must all be same length

In [110]:
# 엑셀로 저장하기
df.to_excel("booking_com.xlsx", encoding='utf-8-sig')

# (참고 코드)

In [83]:
# 호텔 이름
hotel_raw = driver.find_elements_by_css_selector('.sr-hotel__name')
hotel_list = []
for hotel in hotel_raw:
    hotel = hotel.text
    hotel_list.append(hotel)
    
# hotel_list = [ x for x in hotel_list if not x =='']
print(len(hotel_list))
hotel_list

25


['신신호텔 서귀포',
 '롯데호텔 제주',
 '랜딩관 제주신화월드 호텔앤리조트',
 '서머셋 제주신화월드',
 '메종 글래드 제주',
 '신화관 제주신화월드 호텔 앤 리조트',
 '메리어트관 제주신화월드 호텔앤리조트',
 '그랜드 하얏트 제주',
 '롯데시티호텔 제주',
 '브라운 스위트 호텔 & 리조트',
 '호텔 휘슬락',
 '라마다 프라자 호텔',
 '그라벨 호텔 제주',
 '호텔난타 제주',
 '코업시티호텔 하버뷰',
 '휘닉스 제주 섭지코지',
 '해비치 호텔 & 리조트',
 '골든 데이지 호텔',
 '퍼스트70 호텔',
 '그랜드 조선 제주',
 '더 그랜드 섬오름',
 '코업시티호텔 성산',
 '제주 센트럴 시티호텔',
 '위 호텔 제주',
 '신신호텔 제주공항']

In [84]:
# url 
url_list=[]
url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")

for url in url_raw:
    url = url.get_attribute('href')
    url_list.append(url)
    
# url_list = [ x for x in url_list if not x =='']
print(len(url_list))
url_list

25


['https://www.booking.com/hotel/kr/sinsinhotel-seogwipo.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=8511f825a3887039737ca4f646a464e1&all_sr_blocks=696587412_287464310_1_2_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4170&dest_type=region&group_adults=1&group_children=0&hapos=1&highlighted_blocks=696587412_287464310_1_2_0&hpos=1&no_rooms=1&req_adults=1&req_children=0&room1=A&sr_order=popularity&sr_pri_blocks=696587412_287464310_1_2_0__4393400&srepoch=1620707855&srpvid=1da1208719b0009d&ucfs=1&from=searchresults&tpi_r=2#hotelTmpl',
 'https://www.booking.com/hotel/kr/lotte-jeju.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=8511f825a3887039737ca4f646a464e1&all_sr_blocks=23513931_235897805_2_42_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4

In [34]:
url_raw = driver.find_elements_by_css_selector(".sr_item_photo_link.sr_hotel_preview_track")
url_raw[0].get_attribute('href')

'https://www.booking.com/hotel/kr/noblesse-jeju.ko.html?aid=397648&label=nav105jc-1FCAEoggI46AdIM1gDaH2IAQGYARe4AQfIAQzYAQHoAQH4AQyIAgGoAgO4Aov74IQGwAIB0gIkN2VjZDJjNjktMjAxZS00NjBlLTg3ZWQtY2JhMjY2MDljMmM42AIG4AIB&sid=a8969e8a7e83a3dfff572abb1c6df87a&all_sr_blocks=148081801_114185930_2_0_0&checkin=2021-05-15&checkout=2021-05-16&dest_id=4170&dest_type=region&group_adults=1&group_children=0&hapos=251&highlighted_blocks=148081801_114185930_2_0_0&hpos=1&no_rooms=1&req_adults=1&req_children=0&room1=A&sr_order=popularity&sr_pri_blocks=148081801_114185930_2_0_0__5400000&srepoch=1620699854&srpvid=b14010e61385012c&ucfs=1&from=searchresults&tpi_r=2#hotelTmpl'

In [105]:
# 별점 가져오기
rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
rating_list = []
for rating in rating_raw:
    if '\n' in rating.text:
        rating = rating.text.split('\n')[0]
    else:
        rating = rating.text
    rating_list.append(rating)
    
# rating_list = [ x for x in rating_list if not x =='']
print(len(rating_list))
rating_list

25


['7.8',
 '9.4',
 '8.4',
 '7.1',
 '',
 '8.0',
 '7.5',
 '8.1',
 '8.5',
 '8.2',
 '7.8',
 '9.5',
 '6.8',
 '7.7',
 '8.9',
 '8.7',
 '5.8',
 '8.8',
 '9.6',
 '6.3',
 '7.6',
 '9.1',
 '8.9',
 '7.7',
 '9.2']

In [44]:
rating_raw = driver.find_elements_by_css_selector('.sr_item_review_block')
for i in rating_raw:    
    print(i.text, '\n')

6.6
이용 후기 평점
13개 이용 후기
새창에서 열기 

8.2
매우 좋음
160개 이용 후기
새창에서 열기 

7.2
좋음
13개 이용 후기
새창에서 열기 

6.9
이용 후기 평점
9개 이용 후기
새창에서 열기 

9.8
강력 추천
155개 이용 후기
새창에서 열기
위치 9.6
새창에서 열기 

8.0
매우 좋음
159개 이용 후기
새창에서 열기 

8.4
매우 좋음
5개 이용 후기
새창에서 열기
위치 9.5
새창에서 열기 

6.6
이용 후기 평점
51개 이용 후기
새창에서 열기 

7.8
좋음
31개 이용 후기
새창에서 열기 

6.7
이용 후기 평점
10개 이용 후기
새창에서 열기 

8.1
매우 좋음
92개 이용 후기
새창에서 열기 

7.9
좋음
12개 이용 후기
새창에서 열기 

8.6
우수함
28개 이용 후기
새창에서 열기 

8.1
매우 좋음
20개 이용 후기
새창에서 열기 

9.4
최고
113개 이용 후기
새창에서 열기 

 

7.5
좋음
10개 이용 후기
새창에서 열기 

8.2
매우 좋음
142개 이용 후기
새창에서 열기 

9.0
최고
10개 이용 후기
새창에서 열기 

7.8
좋음
14개 이용 후기
새창에서 열기 

8.1
매우 좋음
7개 이용 후기
새창에서 열기 

7.6
좋음
12개 이용 후기
새창에서 열기 

6.6
이용 후기 평점
45개 이용 후기
새창에서 열기 

7.6
좋음
7개 이용 후기
새창에서 열기 

7.3
좋음
22개 이용 후기
새창에서 열기 



In [90]:
# 가격 가져오기
price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
price_list = []
for price in price_raw:
    if price.text.split('\n') != ['']:
        price = price.text.split('\n')
        price = int([ x for x in price if x != '1박, 성인 1명'][0].split(' ')[-1].replace('₩', '').replace(',', ''))
        price_list.append(price)
price_list = [x for x in price_list if not x =='']
print(len(price_list))
price_list

25


[43934,
 514552,
 176411,
 660000,
 175500,
 285474,
 330000,
 552420,
 198000,
 45710,
 91626,
 175000,
 130000,
 89100,
 48599,
 450000,
 580800,
 49754,
 63000,
 627000,
 193600,
 60699,
 52692,
 277041,
 51120]

In [89]:
price_raw = driver.find_elements_by_css_selector('.roomPrice.sr_discount')
for i in price_raw:
    print(i.text.split('\n'))    

['1박, 성인 1명', '₩97,628 ₩43,934', '기존 요금 ₩97,628 현재 요금 ₩43,934', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩514,552', '요금 ₩514,552', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['']
['₩176,411', '세금 및 기타 요금 포함']
['1박, 성인 1명', '₩660,000', '요금 ₩660,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩270,164 ₩175,500', '기존 요금 ₩270,164 현재 요금 ₩175,500', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['']
['₩285,474', '세금 및 기타 요금 포함']
['1박, 성인 1명', '₩330,000', '요금 ₩330,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩552,420', '요금 ₩552,420', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩198,000', '요금 ₩198,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩45,710', '요금 ₩45,710', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['']
['₩91,626', '세금 및 기타 요금 포함']
['1박, 성인 1명', '₩205,787 ₩175,000', '기존 요금 ₩205,787 현재 요금 ₩175,000', '세금 및 기타 요금 포함', '예약 가능 옵션 보기', '새창에서 열기']
['1박, 성인 1명', '₩200,121 ₩130,000', '기존 요금 ₩200,121 현재 요금 ₩130,000

In [91]:
# 방 정보
room_raw = driver.find_elements_by_css_selector(".room_link")
room_list = []
for room in room_raw:
    room_list.append(room.text.split('\n')[0].replace(' -','').strip())
    
room_list = [ x for x in room_list if not x =='']
print(len(room_list))
room_list

25


['Standard Ondol Room(Single use)',
 '한실 패밀리룸',
 '수페리어 트윈',
 '패밀리 스위트룸 한실 (온돌)',
 '디럭스 트윈룸',
 '수페리어 트윈',
 '디럭스 트윈룸',
 '객실 킹침대 1개, 도시 전망',
 '스탠다드 트윈룸',
 '스탠다드 트윈룸',
 '디럭스 더블룸',
 '수페리어 트윈룸 산 전망',
 '디럭스 트윈룸 바다 전망',
 '디럭스 트윈룸',
 '마운틴 뷰 더블룸',
 '로얄룸',
 '[리조트] 스튜디오 트윈 빌리지',
 '수페리어 더블룸',
 '수페리어 트윈룸',
 '디럭스 트윈룸 산의 전망',
 '수페리어 더블룸 신관',
 '스탠다드 더블룸',
 '수페리어 더블룸',
 '트윈룸',
 '체크인 시 객실 배정 주차 불가']